# <font color='blue'> **HW4 Dataset 3**</font>

In [9]:
#install.packages("MLmetrics")
#install.packages("gbm")
library(caret)
library(glmnet)
library(CVXR)
library(MLmetrics)
library(data.table)
library(glmnet)
library(gbm)
library(tidyverse)
library(readxl)

### <font color='red'> **Dataset 3: Cardiotocography Data Set**</font>

Data Set Information:

2126 fetal cardiotocograms (CTGs) were automatically processed and the respective diagnostic features measured. The CTGs were also classified by three expert obstetricians and a consensus classification label assigned to each of them. Classification was both with respect to a morphologic pattern (A, B, C. ...) and to a fetal state (N, S, P). Therefore the dataset can be used either for 10-class or 3-class experiments.


Attribute Information:

LB - FHR baseline (beats per minute)
AC - # of accelerations per second
FM - # of fetal movements per second
UC - # of uterine contractions per second
DL - # of light decelerations per second
DS - # of severe decelerations per second
DP - # of prolongued decelerations per second
ASTV - percentage of time with abnormal short term variability
MSTV - mean value of short term variability
ALTV - percentage of time with abnormal long term variability
MLTV - mean value of long term variability
Width - width of FHR histogram
Min - minimum of FHR histogram
Max - Maximum of FHR histogram
Nmax - # of histogram peaks
Nzeros - # of histogram zeros
Mode - histogram mode
Mean - histogram mean
Median - histogram median
Variance - histogram variance
Tendency - histogram tendency
CLASS - FHR pattern class code (1 to 10)
NSP - fetal state class code (N=normal; S=suspect; P=pathologic)


Relevant Papers:

Ayres de Campos et al. (2000) SisPorto 2.0 A Program for Automated Analysis of Cardiotocograms. J Matern Fetal Med 5:311-318



<font color='purple'>I focused only NSP column. Therefore I excluded CLASS column.

In [2]:
fitControl = trainControl(method = "cv", number = 10)

In [3]:
glm_grid = expand.grid(alpha = 1, lambda = c(0,0.5,1))
dt_grid = expand.grid(cp = c(0.01,0.05,0.1))
rf_grid = expand.grid( mtry = c(3,5,7))
sgb_grid = expand.grid(n.trees = c(30, 50,100), shrinkage = c(0.05,0.1), interaction.depth = c(10, 20), n.minobsinnode = 10)

In [11]:
data1 = read_excel("CTG.xlsx")
data1
dim(data1)

LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
120,0.000,0.000,0.000,0.000,0,0.000,73,0.5,43,...,62,126,2,0,120,137,121,73,1,2
132,0.006,0.000,0.006,0.003,0,0.000,17,2.1,0,...,68,198,6,1,141,136,140,12,0,1
133,0.003,0.000,0.008,0.003,0,0.000,16,2.1,0,...,68,198,5,1,141,135,138,13,0,1
134,0.003,0.000,0.008,0.003,0,0.000,16,2.4,0,...,53,170,11,0,137,134,137,13,1,1
132,0.007,0.000,0.008,0.000,0,0.000,16,2.4,0,...,53,170,9,0,137,136,138,11,1,1
134,0.001,0.000,0.010,0.009,0,0.002,26,5.9,0,...,50,200,5,3,76,107,107,170,0,3
134,0.001,0.000,0.013,0.008,0,0.003,29,6.3,0,...,50,200,6,3,71,107,106,215,0,3
122,0.000,0.000,0.000,0.000,0,0.000,83,0.5,6,...,62,130,0,0,122,122,123,3,1,3
122,0.000,0.000,0.002,0.000,0,0.000,84,0.5,5,...,62,130,0,0,122,122,123,3,1,3
122,0.000,0.000,0.003,0.000,0,0.000,86,0.3,6,...,62,130,1,0,122,122,123,1,1,3


[1] 2126   22

In [12]:
set.seed(1)
trainIndex = createDataPartition(data1$NSP, p = 0.7, list = FALSE)
data1_train_1 = data1[trainIndex, ]
data1_test_1 = data1[-trainIndex, ]

In [14]:
set.seed(2)
data1_glm_model <- train(NSP ~ ., data = data1_train_1, method = "glmnet", trControl = fitControl, tuneGrid = glm_grid)
data1_glm_model

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."

glmnet 

1489 samples
  21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1340, 1340, 1340, 1340, 1340, 1340, ... 
Resampling results across tuning parameters:

  lambda  RMSE       Rsquared   MAE      
  0.0     0.3855612  0.6035901  0.2728190
  0.5     0.6102455        NaN  0.4704885
  1.0     0.6102455        NaN  0.4704885

Tuning parameter 'alpha' was held constant at a value of 1
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were alpha = 1 and lambda = 0.

In [16]:
data1_dt_model <- train(NSP ~ ., data = data1_train_1, method = "rpart", trControl = fitControl, tuneGrid = dt_grid)
data1_dt_model

CART 

1489 samples
  21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1340, 1340, 1341, 1340, 1340, 1340, ... 
Resampling results across tuning parameters:

  cp    RMSE       Rsquared   MAE      
  0.01  0.3250231  0.7050363  0.1465327
  0.05  0.3889633  0.5860499  0.1986367
  0.10  0.4137681  0.5312993  0.2188365

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was cp = 0.01.

In [17]:
data1_rf_model = train(NSP ~ ., data = data1_train_1, method = "rf", trControl = fitControl,tuneGrid = rf_grid)
data1_rf_model

Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in randomForest.default(x, y, 

Random Forest 

1489 samples
  21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1341, 1340, 1340, 1340, 1340, 1340, ... 
Resampling results across tuning parameters:

  mtry  RMSE       Rsquared   MAE      
  3     0.2466033  0.8415650  0.1217866
  5     0.2412926  0.8460644  0.1118235
  7     0.2397466  0.8470367  0.1067509

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 7.

In [18]:
data1_sgb_model = train(NSP ~ ., data = data1_train_1, method = "gbm", trControl = fitControl, tuneGrid = sgb_grid)
data1_sgb_model

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.3550             nan     0.0500    0.0245
     2        0.3310             nan     0.0500    0.0240
     3        0.3058             nan     0.0500    0.0231
     4        0.2832             nan     0.0500    0.0205
     5        0.2633             nan     0.0500    0.0205
     6        0.2456             nan     0.0500    0.0171
     7        0.2310             nan     0.0500    0.0150
     8        0.2153             nan     0.0500    0.0144
     9        0.2030             nan     0.0500    0.0113
    10        0.1906             nan     0.0500    0.0107
    20        0.1119             nan     0.0500    0.0038
    40        0.0587             nan     0.0500    0.0006
    60        0.0434             nan     0.0500    0.0002
    80        0.0365             nan     0.0500    0.0000
   100        0.0318             nan     0.0500   -0.0000

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1       

Stochastic Gradient Boosting 

1489 samples
  21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1340, 1340, 1340, 1340, 1340, 1341, ... 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  RMSE       Rsquared   MAE      
  0.05       10                  30      0.3036729  0.8019344  0.1975560
  0.05       10                  50      0.2671544  0.8157765  0.1409165
  0.05       10                 100      0.2476747  0.8338014  0.1210133
  0.05       20                  30      0.2920908  0.8209625  0.1878653
  0.05       20                  50      0.2534693  0.8343715  0.1319651
  0.05       20                 100      0.2363630  0.8476361  0.1150231
  0.10       10                  30      0.2582691  0.8226184  0.1294618
  0.10       10                  50      0.2486254  0.8317156  0.1238296
  0.10       10                 100      0.2408531  0.8411712  0.1233320
  0.10       20                  30      

In [19]:
data1_glm_model$bestTune
data1_dt_model$bestTune
data1_rf_model$bestTune
data1_sgb_model$bestTune

alpha,lambda
1,0


cp
0.01


,mtry
3,7


,n.trees,interaction.depth,shrinkage,n.minobsinnode
6,100,20,0.05,10


The best parameters for 4 models are shown above.

In [20]:
glm_RMSE = min(data1_glm_model$result$RMSE)
dt_RMSE = min(data1_dt_model$result$RMSE)
rf_RMSE = min(data1_rf_model$result$RMSE)
sgb_RMSE = min(data1_sgb_model$result$RMSE)
data.table(glm_RMSE,dt_RMSE,rf_RMSE,sgb_RMSE)

glm_RMSE,dt_RMSE,rf_RMSE,sgb_RMSE
0.3855612,0.3250231,0.2397466,0.236363


In [21]:
glm_MAE = min(data1_glm_model$result$MAE)
dt_MAE = min(data1_dt_model$result$MAE)
rf_MAE = min(data1_rf_model$result$MAE)
sgb_MAE = min(data1_sgb_model$result$MAE)
data.table(glm_MAE,dt_MAE,rf_MAE,sgb_MAE)

glm_MAE,dt_MAE,rf_MAE,sgb_MAE
0.272819,0.1465327,0.1067509,0.1150231


As seen the results above, SGB and RF models have the best MAE and RMSE results, and results are almost same. Therefore 2 models can be used for train data. Howecer RF is better MAE result than SGB. RF can be more accurate result.

In [22]:
data1_glm_predict <- predict(data1_glm_model,data1_test_1)   
data1_dt_predict <- predict(data1_dt_model,data1_test_1)   
data1_rf_predict <- predict(data1_rf_model,data1_test_1)   
data1_sgb_predict <- predict(data1_sgb_model,data1_test_1)  

In [24]:
glm_Test_RMSE = RMSE(data1_test_1$NSP,data1_glm_predict)
dt_Test_RMSE = RMSE(data1_test_1$NSP,data1_dt_predict)
rf_Test_RMSE = RMSE(data1_test_1$NSP,data1_rf_predict)
sgb_Test_RMSE = RMSE(data1_test_1$NSP,data1_sgb_predict)
RSME_result = data.table(glm_Test_RMSE,dt_Test_RMSE,rf_Test_RMSE,sgb_Test_RMSE)
RSME_result

glm_Test_RMSE,dt_Test_RMSE,rf_Test_RMSE,sgb_Test_RMSE
0.4050959,0.3482782,0.2603071,0.2706419


As same as train dataset, RMSE value is the best for RF model. Therefore RF Model with MTRY=7 is the best for this dataset.